In [1]:
import numpy as np
import re
import random

import tensorflow as tf
import sklearn as sk

import Functions as fn
from DS import DS
from Set import pool
from Iterator import Iterator

In [2]:
#Dataset = fn.first_time_load()
#Dataset.write_texts()
#Dataset.write_labels()

In [3]:
Dataset = pool()
Dataset.load_texts()
Dataset.load_labels()

Texts loaded
Labels loaded


In [4]:
#print('Number of Texts: ', Dataset.size)
#print('Number of 2007 Smoking Challenge texts: ', Dataset.number_of(challenge='2007 Smoking Challenge'))
#print('Number of 2008 ObesityChallenge texts: ', Dataset.number_of(challenge='2008 Obesity Challenge'))
#print('Number of 2009 Medication Challenge texts: ', Dataset.number_of(challenge='2009 Medication Challenge'))
#print('Number of 2010 Relations Challenge texts: ', Dataset.number_of(challenge='2010 Relations Challenge'))
#print('Number of 2011 Coreference Challenge texts: ', Dataset.number_of(challenge='2011 Coreference Challenge'))
#print('Number of 2012 Temporal Relations Challenge texts: ', Dataset.number_of(challenge='2012 Temporal Relations Challenge'))
#print('Number of Train Texts: ', Dataset.number_of(stage='train'))
#print('Number of Test Texts: ', Dataset.number_of(stage='test'))
#print('Number of Labeled Texts: ', Dataset.number_of(labelled='yes'))
#print('Number of Initially Labeled Texts: ', Dataset.number_of(labelled='yes', label_type='train'))
#print('Number of Competitor Labeled Texts Texts: ', Dataset.number_of(labelled='yes', label_type='test'))

In [5]:
Dataset.process_for_embedding()
sentences = Dataset.get_sentences()

In [6]:
from gensim.models import Word2Vec

C:\Users\Luka\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
model = Word2Vec.load('W2V')

In [8]:
#model = Word2Vec(sentences, min_count=1, size=100)

In [9]:
#model.most_similar("prozac", topn=10)

In [10]:
targets = ['medications', 'dosages', 'modes', 'frequencies', 'durations', 'reasons']
target_dict = {label: words for (label, words) in zip(targets, list(fn.label_words(Dataset)))}

Number of: m=928, do=119, mo=76, f=79, du=80, r=482


In [11]:
#fn.visualise(model, sentences, [target_dict['medications']], 1000, 'Medications in top 1000 words')

In [12]:
#print_set = set()
#for case in Dataset.get_DS(labelled='yes').data:
#    for term in re.finditer(r'm="[a-z0-9 ]+"', case.raw_labels):
#        if term.group()[3:-1] != 'nm': print_set.add(term.group()[3:-1])
#print(*print_set, sep='\n')

In [13]:
#for target in targets:
#    for reps in [True, False]:
#        target_size = len(target_dict[target])
#        sets = fn.get_traintest(model, target_dict[target], target_size * 5, target_size, 10, 50, reps, reps)
#        fn.model_1(sets[0], sets[1], sets[2], sets[3], target=target, repetitions='yes' if reps else 'no')

In [14]:
#fn.write_emb('testemb', model)

In [15]:
#test_model = fn.load_emb('testemb')

In [16]:
#model.save('W2V')

In [17]:
labelled_cases = Dataset.get_DS(labelled='yes')

In [18]:
len(labelled_cases.data)

258

In [19]:
for case in labelled_cases.data:
    text = case.raw_text
    text = text.split('\n')
    for i in range(len(text)):
        text[i] = text[i].strip('.')
        text[i] = re.sub(r'\d+', '<NUM>', text[i])
        text[i] = re.sub(r'([A-Za-z]):', r'\1', text[i])
        text[i] = re.sub(r'Dr.', 'Dr', text[i])
        text[i] = re.sub(r'Mr.', 'Mr', text[i])
        text[i] = re.sub(r'\. ([A-Z])', r'. A\1', text[i])
        text[i] = re.sub(r'\. [A-Z]', ' ', text[i])
        text[i] = text[i].lower()
        text[i] = text[i].split()
    case.token_text = text
    
    indeces = []
    for med in re.finditer(r'm="[^"]*" \d+:\d+ \d+:\d+', case.raw_labels):
        indeces.append([[int(a) for a in b.split(':')] for b in med.group().split()[-2:]])
    indeces.sort()
    indeces.append([[0,0],[0,0]])
    
    truth = []
    c = 0
    inside =  False
    for i in range(len(text)):
        for j in range(len(text[i])):
            if inside:
                if i+1 < indeces[c][1][0]: truth.append([1,0])
                elif i+1 == indeces[c][1][0]:
                    if j < indeces[c][1][1]: truth.append([1,0])
                    elif j == indeces[c][1][1]:
                        truth.append([1,0])
                        inside = False
                        c += 1
            else:
                if [i+1, j] == indeces[c][0]:
                    truth.append([1,0])
                    if [i+1, j] == indeces[c][1]: c += 1
                    else: inside = True
                else: truth.append([0,1])
    case.test_labels = truth
    
    case.test_text = [word for sentence in case.token_text for word in sentence]

In [24]:
n = 3
print([labelled_cases.data[n].test_text[a] for a in range(len(labelled_cases.data[n].test_text)) if labelled_cases.data[n].test_labels[a] == [1, 0]])

['vancomycin', 'antibiotics', 'antibiotics', 'antibiotics', 'colace', 'folate', 'gemfibrozil', 'lantus', 'lipitor', 'nephrocaps', 'neurontin', 'phoslo', 'protonix', 'renagel', 'requip', 'coumadin', 'vancomycin', 'unasyn', 'unasyn', 'augmentin', 'vancomycin', 'levofloxacin', 'flagyl', 'vancomycin', 'pain', 'medication', 'as', 'coumadin', 'coumadin', 'heparin', 'coumadin', 'heparin', 'heparin', 'heparin', 'levofloxacin', '<num>mg', '<num>mg', 'gemfibrozil', 'lipitor', 'lipitor', 'b-blocker', 'lisinopril', 'lopressor', 'lantus', 'lantus', 'aspart', 'ketamine', 'fentanyl', 'pca', 'fentanyl', 'pca', 'ketamine', 'oxycontin', 'oxycodone', 'lyrica', 'ativan', 'heparin', 'levofloxacin;', 'coumadin', 'pain', 'medications']


In [48]:
train_set = [model[word] if word in model.wv.vocab else np.zeros(100) for case in labelled_cases.data[:230] for word in case.test_text]
train_labels = [label for case in labelled_cases.data[:230] for label in case.test_labels] 
test_set = [model[word] if word in model.wv.vocab else np.zeros(100) for case in labelled_cases.data[230:] for word in case.test_text]
test_labels = [label for case in labelled_cases.data[230:] for label in case.test_labels]

test_words = [word for case in labelled_cases.data[230:] for word in case.test_text]
print(*[len(a) for a in [train_set, train_labels, test_set, test_labels, test_words]])

260818 260818 31810 31810 31810


In [49]:
print(np.array(train_labels).sum(0)/ len(train_labels))
print(len(train_set))

[ 0.02997876  0.97002124]
260818


In [50]:
while (np.array(train_labels).sum(0)/ len(train_labels))[0] < 0.1:
    for med in target_dict['medications']:
        train_set.append(model[med])
        train_labels.append([1,0])
    print(np.array(train_labels).sum(0)/len(train_labels))

[ 0.03341789  0.96658211]
[ 0.03683273  0.96316727]
[ 0.04022352  0.95977648]
[ 0.04359052  0.95640948]
[ 0.04693398  0.95306602]
[ 0.05025414  0.94974586]
[ 0.05355125  0.94644875]
[ 0.05682555  0.94317445]
[ 0.06007727  0.93992273]
[ 0.06330665  0.93669335]
[ 0.06651391  0.93348609]
[ 0.06969929  0.93030071]
[ 0.072863  0.927137]
[ 0.07600526  0.92399474]
[ 0.07912629  0.92087371]
[ 0.08222632  0.91777368]
[ 0.08530554  0.91469446]
[ 0.08836417  0.91163583]
[ 0.09140241  0.90859759]
[ 0.09442046  0.90557954]
[ 0.09741854  0.90258146]
[ 0.10039682  0.89960318]


In [51]:
print(np.array(train_labels).sum(0)/ len(train_labels))

[ 0.10039682  0.89960318]


In [52]:
node_count = 10
input_size = 100
output_size = 2
batch_size = 50
epochs = 10
report_percentage = 10

In [53]:
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.05)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    node_count = node_count

    x = tf.placeholder(tf.float32, shape=[None, input_size])
    y_ = tf.placeholder(tf.float32, shape=[None, output_size])

    # Define the first layer here
    W = weight_variable([input_size, node_count])
    b = bias_variable([node_count])
    h = tf.nn.sigmoid(tf.matmul(x, W) + b)

    # Use dropout for this layer (should you wish)
    # keep_prob = tf.placeholder(tf.float32)
    # h_drop = tf.nn.dropout(h1, keep_prob)

    # Define the second layer here
    # W2 = weight_variable([node_count_1, node_count_2])
    # b2 = bias_variable([node_count_2])
    # h2 = tf.nn.sigmoid(tf.matmul(h, W) + b)

    # Define the output layer here
    V = weight_variable([node_count, output_size])
    c = bias_variable([output_size])
    y = tf.nn.softmax(tf.matmul(h, V) + c)

    # We'll use the cross entropy loss function
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))

    # And classification accuracy
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # And the Adam optimiser
    train_step = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cross_entropy)

    # Start a tf session and run the optimisation algorithm
    sess = tf.Session()
    # sess.run(tf.initialize_all_variables())
    sess.run(tf.global_variables_initializer())

    training = Iterator(train_set, train_labels)
    mark = (epochs * (len(train_set) // batch_size) * report_percentage) // 100
    N = 0

    #print("Target: %s \tRepetitions: %s" % (target, repetitions))
    while training.epochs < epochs:
        trd, trl = training.next_batch(batch_size)
        if N % mark == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: trd, y_: trl})
            test_accuracy = sess.run(accuracy, feed_dict={x: test_set, y_: test_labels})
            print("Progress: %d%% \tTraining Accuracy: %f\t Test accuracy: %f" % (N * report_percentage// mark, train_accuracy, test_accuracy))
        sess.run(train_step, feed_dict={x: trd, y_: trl})
        N += 1

    print("Final Test Accuracy: %f\n" % (sess.run(accuracy, feed_dict={x: test_set, y_: test_labels})))

Progress: 0% 	Training Accuracy: 0.880000	 Test accuracy: 0.970387
Progress: 10% 	Training Accuracy: 1.000000	 Test accuracy: 0.972587
Progress: 20% 	Training Accuracy: 1.000000	 Test accuracy: 0.972713
Progress: 30% 	Training Accuracy: 0.980000	 Test accuracy: 0.972996
Progress: 40% 	Training Accuracy: 0.960000	 Test accuracy: 0.974316
Progress: 50% 	Training Accuracy: 0.960000	 Test accuracy: 0.972619
Progress: 60% 	Training Accuracy: 0.980000	 Test accuracy: 0.973813
Progress: 70% 	Training Accuracy: 1.000000	 Test accuracy: 0.974505
Progress: 80% 	Training Accuracy: 0.900000	 Test accuracy: 0.974505
Progress: 90% 	Training Accuracy: 1.000000	 Test accuracy: 0.974882
Progress: 100% 	Training Accuracy: 0.980000	 Test accuracy: 0.973562
Final Test Accuracy: 0.973530



In [54]:
pred = tf.argmax(y, 1)
res = sess.run(pred, feed_dict={x: test_set, y_:test_labels})
tru = np.argmax(test_labels, 1)

In [55]:
sk.metrics.f1_score(tru, res, pos_label=0, average='binary')

0.62511130899376666

In [56]:
[(test_words[a], res[a], tru[a]) for a in range(len(test_words)) if res[a] == 0 and tru[a] == 1]

[('products', 0, 1),
 ('niacin', 0, 1),
 ('sodium', 0, 1),
 ('hydration', 0, 1),
 ('hydration', 0, 1),
 ('ace', 0, 1),
 ('requring', 0, 1),
 ('bb', 0, 1),
 ('hydration', 0, 1),
 ('acei', 0, 1),
 ('hydration', 0, 1),
 ('insulin', 0, 1),
 ('insulin', 0, 1),
 ('synthroid', 0, 1),
 ('levothyroxine', 0, 1),
 ('sodium', 0, 1),
 ('milk', 0, 1),
 ('magnesia', 0, 1),
 ('magnesium', 0, 1),
 ('reglan', 0, 1),
 ('metoclopramide', 0, 1),
 ('hcl', 0, 1),
 ('zofran', 0, 1),
 ('ondansetron', 0, 1),
 ('hcl', 0, 1),
 ('post-...', 0, 1),
 ('quinapril', 0, 1),
 ('kcl', 0, 1),
 ('quinapril', 0, 1),
 ('hcl', 0, 1),
 ('potassium', 0, 1),
 ('chloride', 0, 1),
 ('quinapril', 0, 1),
 ('hcl', 0, 1),
 ('potassium', 0, 1),
 ('chloride', 0, 1),
 ('simethicone', 0, 1),
 ('multivitamin', 0, 1),
 ('multivi...', 0, 1),
 ('tigan', 0, 1),
 ('hcl', 0, 1),
 ('tigan', 0, 1),
 ('hcl', 0, 1),
 ('morphine', 0, 1),
 ('aspirin', 0, 1),
 ('motrin', 0, 1),
 ('aleve', 0, 1),
 ('advil', 0, 1),
 ('softener', 0, 1),
 ('antibiotics', 0

In [58]:
tp = len([a for a in range(len(tru)) if (res[a] == 0) and (tru[a] == 0)])
tn = len([a for a in range(len(tru)) if (res[a] == 1) and (tru[a] == 1)])
fp = len([a for a in range(len(tru)) if (res[a] == 0) and (tru[a] == 1)])
fn = len([a for a in range(len(tru)) if (res[a] == 1) and (tru[a] == 0)])
tp, tn, fp, fn

(702, 30266, 602, 240)

In [59]:
print((2*tp)/(2*tp + fn +fp))

0.6251113089937667
